In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
gau_rages_page = requests.get("https://finalfantasy.wikia.com/wiki/Rage_(Final_Fantasy_VI_command)")
soup = BeautifulSoup(gau_rages_page.content, "html.parser")

In [4]:
list_of_rages_header = soup.find("span", id="List_of_Rages")
rage_table = list_of_rages_header.findNext("table")

In [36]:
all_table_th = rage_table.findAll("th")
base_link = "https://finalfantasy.wikia.com"
ff6_url = "/wiki/Final_Fantasy_VI"
enemy_abilities_link = "https://finalfantasy.wikia.com/wiki/List_of_Final_Fantasy_VI_enemy_abilities"

abilities = requests.get(enemy_abilities_link)
abilities_soup = BeautifulSoup(abilities.content, "html.parser")

# For specific spells, do certain crawling exceptions
specific_exceptions = set()
specific_exceptions.add("Blizzaga_(ability)")

misspells_dict = {}
misspells_dict["Incisors"] = "Incisor"
misspells_dict["Landslide"] = "Land_Slide"

visited_id = set()
# Skip these specific spells
visited_id.add("Tornado_(ability)")
visited_id.add("Break_(ability)")

output_file = "spell_power.csv"
headers = ["Ability Name", "Magic Power", "Description"]

df = pd.DataFrame(columns=headers)
    
for i in range(7, len(all_table_th), 3):
    cur_th = all_table_th[i]
    a_link = cur_th.find("a").get('href')
    ability_link = "".join((base_link, a_link))

    # For now, only check the enemy abilities link
    if (enemy_abilities_link in ability_link):
        id_1 = ability_link.split("#")[1]
        # If misspelled
        if id_1 in misspells_dict:
            id_1 = misspells_dict[id_1]

        next_row = []
        if id_1 not in visited_id:
            visited_id.add(id_1)

            span_found = abilities_soup.find("span", id=id_1)
            # If we cannot find the span, we need to manually add it to misspells_dict
            if span_found is None:
                print(ability_link)

            next_td = str(span_found.findNext("td"))
            # if the next td has the word "spell power" in it
            spell_power = 0
            physical_power = 0

            if "spell power" in next_td.lower():
                index_of_spell = next_td.index("Spell Power")
                substr_1 = next_td[index_of_spell - 10:index_of_spell + 25]
                temp_power = []
                # Iterate through substr until you find a digit
                for char in substr_1:
                    if char.isdigit():
                        temp_power.append(char)
                temp_power = int("".join(temp_power))

                next_row.append(id_1)
                next_row.append(temp_power)
                next_row.append(next_td)
                df.loc[len(df.index)] = next_row
                
                s = ",".join((str(id_1), str(temp_power), next_td, "\n"))
#                 file.write(s)
    
    else:
        # Not on the "ability page" page
        temp_page = requests.get(ability_link)
        temp_soup = BeautifulSoup(temp_page.content, "html.parser")
        temp_ff6 = temp_soup.find("a", href=ff6_url, title="Final Fantasy VI")
        
        if temp_ff6 is not None:
            id_1 = ability_link.split("wiki/")[1]
            
            if id_1 not in visited_id:
                visited_id.add(id_1)
                
                if id_1 in specific_exceptions:
                    temp_power = 122
                    p = "<p>Blizzaga is an <a href=\"/wiki/Magic_(Final_Fantasy_VI_command)\" title=\"Magic (Final Fantasy VI \
                         command)\">Attack spell</a> thought by <a href=\"/wiki/Valigarmanda\" title=\"Valigarmanda\">\
                         Valigarmanda</a> at a rate of x1. <a href=\"/wiki/Celes_Chere/Gameplay\" title=\"Celes Chere\
                         /Gameplay\">Celes</a> also learns the spell at level 42. It costs 52 MP to cast, has a hit rate \
                         of 150, an attack power of 122, and is vulnerable to <a href=\"/wiki/Runic\" \
                         title=\"Runic\">Runic</a>. The spell is used by many enemies, the earliest being used by \
                         <a href=\"/wiki/Dullahan_(Final_Fantasy_VI)\" title=\"Dullahan (Final Fantasy VI)\">Dullahan</a>.</p>"
                    next_row.append(id_1)
                    next_row.append(temp_power)
                    next_row.append(p)
                    df.loc[len(df.index)] = next_row
            
                else:
                    # Find the next 3 <p> tags
                    next_p_1 = temp_ff6.find_next("p")
                    next_p_2 = next_p_1.find_next("p")
                    next_p_3 = next_p_2.find_next("p")
                    next_p_list = [next_p_1, next_p_2, next_p_3]

                    next_row = []
                    for p in next_p_list:
                        p = str(p)
                        if "power " in str(p).lower():
                            index_of_power = p.lower().index("power ")
                            # substr_1 = p[index_of_power - 10: index_of_power + 25]
                            substr_1 = p[index_of_power: index_of_power + 25]
                            temp_power = []

                            # Iterate through substr until you find a digit
                            for char in substr_1:
                                if char.isdigit():
                                    temp_power.append(char)
                                    if (len(temp_power)) > 2:
                                        break
                            try:
                                temp_power = int("".join(temp_power))
                            except ValueError:
                                print(df.to_string())
                                print("{}, {}".format(p, substr_1))
                                raise
                            next_row.append(id_1)
                            next_row.append(temp_power)
                            next_row.append(p)
                            df.loc[len(df.index)] = next_row
                            break
                
        # Weird links
        else:
            print("INVALID LINK: {}".format(ability_link))    

INVALID LINK: https://finalfantasy.wikia.com/wiki/Cave-In
INVALID LINK: https://finalfantasy.wikia.com/wiki/Lifebreak
INVALID LINK: https://finalfantasy.wikia.com/wiki/Lifebreak
INVALID LINK: https://finalfantasy.wikia.com/wiki/Lifebreak


In [37]:
# test_url = "https://finalfantasy.wikia.com/wiki/Fira_(ability)#Final_Fantasy_VI"
# ff6_url = "/wiki/Final_Fantasy_VI"
# page = requests.get(test_url)
# delicious_soup = BeautifulSoup(page.content, "html.parser")

# temp_ff6 = delicious_soup.find("a", href=ff6_url, title="Final Fantasy VI")
# p_1 = temp_ff6.find_next("p")
# p_2 = p_1.find_next("p")
# p_3 = p_2.find_next("p")
# # print(p_1)
# # print(p_2)
# # print(p_3)
# list1 = [p_1, p_2, p_3]

# for p in list1:
#     p = str(p)
#     if "power" in str(p):
#         index_of_power = p.index("power")
#         substr_1 = p[index_of_power - 10:index_of_power + 25]
#         print(substr_1)
#         break

df2 = df.sort_values("Magic Power", ascending=False)
df2

,Ability Name,Magic Power,Description
26,Meltdown_(Magic),138,<p>Meltdown is an Attack spell (originally <a ...
5,Shock_(ability),128,<p>Shock is the special command ability of <a ...
38,Blizzaga_(ability),122,"<p>Blizzaga is an <a href=""/wiki/Magic_(Final_..."
30,Firaga_(ability),121,"<p>Firaga is an <a href=""/wiki/Magic_(Final_Fa..."
8,Thundaga_(ability),120,"<p>Thundaga is an <a href=""/wiki/Magic_(Final_..."
16,Metal_Cutter,120,<td>Deals non-elemental magic damage to one or...
19,Quake_(ability),111,<p>Quake is an Attack spell that deals earth-e...
27,Gigavolt,110,<td>Deals Lightning-elemental damage to one or...
3,Holy_(ability),108,"<p>Holy is classified as an <a href=""/wiki/Mag..."
2,Flood_(Ninjutsu),100,"<p>The <a href=""/wiki/List_of_Final_Fantasy_VI..."


In [18]:
df2.to_csv(output_file)

In [28]:
url = "https://finalfantasy.wikia.com/wiki/Blizzaga_(ability)#Final_Fantasy_VI"
r = requests.get(url)
bs = BeautifulSoup(r.content, "html.parser")
bs_ff6 = bs.find("a", href=ff6_url, title="Final Fantasy VI")
b = bs_ff6.find_all_next("p")

for b1 in b:
    f = b1.find("a")
    href1 = f.get("href")
    if "Final_Fantasy_VI" in href1:
        print(b1)
        break

<p>Blizzaga is an <a href="/wiki/Magic_(Final_Fantasy_VI_command)" title="Magic (Final Fantasy VI command)">Attack spell</a> thought by <a href="/wiki/Valigarmanda" title="Valigarmanda">Valigarmanda</a> at a rate of x1. <a href="/wiki/Celes_Chere/Gameplay" title="Celes Chere/Gameplay">Celes</a> also learns the spell at level 42. It costs 52 MP to cast, has a hit rate of 150, an attack power of 122, and is vulnerable to <a href="/wiki/Runic" title="Runic">Runic</a>. The spell is used by many enemies, the earliest being used by <a href="/wiki/Dullahan_(Final_Fantasy_VI)" title="Dullahan (Final Fantasy VI)">Dullahan</a>.
</p>
